In [1]:
from mypackage import scraping_functions as sf

In [2]:
game_url = 'https://www.pro-football-reference.com/boxscores/202309070kan.htm'
df = sf.scrape_game_data(game_url)

Chiefs vs. Lions 
 total plays: 132


In [3]:
df

,Quarter,Time,Down,ToGo,Location,DET,KAN,Detail,EPB,EPA,field_side,yardline,play_start_time,Play_Type,possession,Yardage
1,1,15:00,,,KAN 35,0,0,"Harrison Butker kicks off 65 yards, touchback.",0.000,0.610,KAN,35,0.000000,Special Teams,DET,0.0
2,1,15:00,1,10,DET 25,0,0,David Montgomery up the middle for 7 yards (ta...,0.610,1.010,DET,25,0.000000,Run,DET,7.0
3,1,14:29,2,3,DET 32,0,0,Jared Goff pass incomplete short right,1.010,0.300,DET,32,0.516667,Pass,DET,0.0
4,1,14:25,3,3,DET 32,0,0,Jared Goff pass incomplete deep right intended...,0.300,-1.240,DET,32,0.583333,Pass,DET,0.0
5,1,14:19,4,3,DET 32,0,0,"Jack Fox punts 61 yards, returned by Richie Ja...",-1.240,0.320,DET,32,0.683333,Special Teams,DET,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,4,1:53,2,5,KAN 25,21,20,David Montgomery right guard for 3 yards (tack...,3.710,3.400,KAN,25,58.116667,Run,DET,3.0
137,4,1:47,3,2,KAN 22,21,20,David Montgomery up the middle for 2 yards (ta...,3.400,4.240,KAN,22,58.216667,Run,DET,2.0
139,4,1:42,1,10,KAN 20,21,20,Jared Goff kneels for -1 yards,4.240,3.560,KAN,20,58.300000,Run,DET,-1.0
140,4,1:05,2,11,KAN 21,21,20,Jared Goff kneels for -1 yards,3.560,2.740,KAN,21,58.916667,Run,DET,-1.0


In [ ]:
# Quarter
# play start time (sec)
# Down
# ToGo
# Scores
# TBD on EPB EPA
# yardage
# field_side
# yardline
# adjusted score
# play_type
# possession
# yardage

